# lib-ela: A Notebook Walk-through

Welcome to **lib-ela**! This notebook will guide you through the basics of symbolic solid mechanics modeling with Python.
You'll learn to install the package, explore available material models, run common deformation protocols, visualise results, and export data.


## 1. Installation

If you haven't installed *lib-ela* and *matplotlib* yet, run:
```bash
!pip install lib-ela matplotlib
```
*(In Jupyter, remove the backticks and execute the line in a code cell.)*

In [ ]:
#!pip install lib-ela matplotlib

## 2. Importing and Exploring Material Models

We'll import **lib-ela** and inspect the *NeoHookean* material. Additional Hyperelastic models (e.g. *MooneyRivlin*, *Yeoh*) follow the same methods.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from libela.hyperelastic import NeoHookean
# from libela.hyperelastic import MooneyRivlin, Yeoh, Polynomial

help(NeoHookean)  # show full docstring


## 3. First Example: Uniaxial Stretch

Create a *Neo-Hookean* material and compute the Cauchy stress under uniaxial loading.

In [ ]:
MU, K = 700.0, 2500.0  # shear & bulk moduli [kPa]
neo = NeoHookean(mu=MU, k=K)

lam = np.linspace(0.6, 1.4, 50)
sigma = neo.stress(lam, params=[MU, K],
                   protocol='uniaxial', stress_type='cauchy')

plt.figure(figsize=(6,4))
plt.plot(lam, sigma, label='Neo-Hookean')
plt.xlabel('Stretch λ')
plt.ylabel('Cauchy stress [kPa]')
plt.title('Uniaxial stress–stretch curve')
plt.grid(True)
plt.legend()
plt.show()


## 4. Trying Other Protocols

Compute stresses for *simple shear* and *planar biaxial*.

In [ ]:
# Simple shear
gamma = np.linspace(0, 1.0, 50)
tau_shear = neo.stress(gamma, params=[MU, K],
                       protocol='shear', stress_type='kirchhoff')

plt.figure(figsize=(6,4))
plt.plot(gamma, tau_shear, label='τ12 (Kirchhoff)')
plt.xlabel('Engineering shear γ')
plt.ylabel('Kirchhoff stress [kPa]')
plt.title('Simple-shear response')
plt.grid(True); plt.legend(); plt.show()


In [ ]:
# Planar biaxial: λ1 = λ2 sweep
lam_b = np.linspace(0.9, 1.3, 40)
grid = np.stack([lam_b, lam_b])  # shape (2, N)

sigma_b = neo.stress(grid, params=[MU, K],
                     protocol='biaxial', stress_type='cauchy')[0]

plt.figure(figsize=(6,4))
plt.plot(lam_b, sigma_b, label='σ11 (biaxial)')
plt.xlabel('Stretch λ')
plt.ylabel('Cauchy stress [kPa]')
plt.title('Planar-biaxial response')
plt.grid(True); plt.legend(); plt.show()


## 5. Comparing Materials *(optional)*

When more materials are available, uncomment the following to compare their responses.

In [ ]:
# from libela.hyperelastic import MooneyRivlin
# mr = MooneyRivlin(c1=MU/2, c2=MU/2, k=K)
# sigma_mr = mr.stress(lam, params=[MU/2, MU/2, K],
#                      protocol='uniaxial', stress_type='cauchy')
#
# plt.figure(figsize=(6,4))
# plt.plot(lam, sigma, label='Neo-Hookean')
# plt.plot(lam, sigma_mr, label='Mooney–Rivlin')
# plt.xlabel('λ'); plt.ylabel('σ11 [kPa]')
# plt.title('Material comparison – uniaxial')
# plt.grid(True); plt.legend(); plt.show()


## 6. Saving & Exporting Results

Save curves and raw data for reports or papers.

In [ ]:
np.savetxt('uniaxial_stress_curve.csv',
           np.column_stack((lam, sigma)),
           delimiter=',', header='lambda,sigma11_kPa', comments='')
plt.figure(figsize=(6,4))
plt.plot(lam, sigma)
plt.xlabel('λ'); plt.ylabel('σ11 [kPa]')
plt.title('Uniaxial σ11'); plt.grid(True)
plt.savefig('uniaxial_stress_curve.png', dpi=300)
plt.close()
print('Saved uniaxial_stress_curve.csv & .png')


## 7. Bonus: Interactive Slider *(Jupyter only)*

Use an *ipywidgets* slider to see how changing the shear modulus μ shifts the curve.

In [ ]:
# %matplotlib widget
# from ipywidgets import FloatSlider
# from IPython.display import display
#
# slider = FloatSlider(value=MU, min=100, max=2000, step=100,
#                      description='μ [kPa]', continuous_update=False)
# fig, ax = plt.subplots(figsize=(6,4))
#
# def update(mu_val):
#     ax.clear()
#     neo_tmp = NeoHookean(mu=mu_val, k=K)
#     sigma_tmp = neo_tmp.stress(lam, params=[mu_val, K],
#                                protocol='uniaxial', stress_type='cauchy')
#     ax.plot(lam, sigma_tmp)
#     ax.set_xlabel('λ'); ax.set_ylabel('σ11 [kPa]')
#     ax.set_title(f'Neo-Hookean, μ = {mu_val:.0f} kPa'); ax.grid(True)
#
# slider.observe(lambda ch: update(ch['new']), names='value')
# display(slider); update(MU)


## 8. Next Steps

- Explore `libela.hyperelastic.operations` for deformation-gradient details.
- Fit your own experimental data (module `libela.fitting`, coming soon).
- Dive deeper with the *Theory* and *Tutorials* sections on the documentation site.
